In [29]:
# %pip uninstall python-dotenv
# %pip install python-dotenv


In [30]:
# import some basic python libaries (operating system, regular expressions, time)
import os
import re
import time
from datetime import datetime
import random
import requests
# import some selenium components to rememeneber by name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

from selenium_stealth import stealth

# Import Load login credentials from .env file, make sure to pip install python-dotenv
#import mongoDB, we may add additional databases in the future for better 

# from dotenv import find_dotenv,load_dotenv
# load_dotenv(find_dotenv())

from pymongo import MongoClient

from bs4 import BeautifulSoup


# Setup

In [31]:
# Create a MongoClient to the running mongod instance
client = MongoClient("mongodb://localhost:27017")
db = client["my_database"]
#Delete all documents in the collection before we start 
collection_linkedin = db['opportunities_linkedin']
result = collection_linkedin.delete_many({})
print(result.deleted_count, " documents deleted.")
url = r"https://www.linkedin.com/login"
# url = r"https://www.linkedin.com/jobs/search/?currentJobId=3947699666&distance=25&f_E=1&geoId=101174742&keywords=software%20intern&origin=JOBS_HOME_KEYWORD_HISTORY&refresh=true"

#current tie for log name
timestamp = datetime.now().strftime('%Y_%m_%d_%H-%M-%S')

# Load the .env file
# email = os.getenv("LINKEDIN_EMAIL")
# password = os.getenv("LINKEDIN_PASSWORD")

0  documents deleted.


# Data Ingestion

In [32]:
# Open web page
# Add user-agent to the webdriver options
user_agents = [
    # Your list of user agents goes here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36", 
]
user_agent = random.choice(user_agents)

options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

# Initialize the WebDriver with options
driver = webdriver.Chrome(options=options)

# more attempts
# Adding argument to disable the AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 

# Exclude the collection of enable-automation switches 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 

# Turn-off userAutomationExtension 
options.add_experimental_option("useAutomationExtension", False) 

# Apply stealth settings to the driver
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )
#initialize webdriver
driver = webdriver.Chrome(options=options)
actions = ActionChains(driver)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
driver.get(url)



time.sleep(3)
# Login
email_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "username")))
email_box.send_keys("simonandorsimon@gmail.com")
time.sleep(1)
password_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "password")))
password_box.send_keys("Howthefuckdoyouknowmypasswordlittle1")
time.sleep(1)
sign_in_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Sign in']")))
sign_in_button.click()
# Open search link then ingest all jobs
time.sleep(3)
search_url = r"https://www.linkedin.com/jobs/search/?currentJobId=3941700835&f_E=1&geoId=101174742&keywords=software%20intern%20development&location=Canada&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true"
driver.get(search_url)
time.sleep(3)

#scroll to page right button
page_right_button = driver.find_element(By.ID, 'ember268')
actions.move_to_element(page_right_button).perform()

# card_box =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "strong")))
cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'job-card-container--clickable')]")
# print(titles)
# Open search link then ingest all jobs
print(len(cards))
for title in cards:
    print(title.text)
    title.click()
    time.sleep(3)



time.sleep(5)
# driver.quit()

12
étudiant Student
étudiant Student with verification
Covia
Mirabel, QC
Viewed
Promoted
Stagiaire en développement Web
Stagiaire en développement Web with verification
Petal
Montréal-Ouest, QC (On-site)
Actively recruiting
Viewed
Promoted
C-DS-450 Stagiaire, Technicien(ne) simulateur
C-DS-450 Stagiaire, Technicien(ne) simulateur with verification
CAE
Moose Jaw, SK
Actively recruiting
Promoted
1 applicant
Student Mechanic
Student Mechanic with verification
Prysmian
Quebec, Canada (On-site)
Actively recruiting
Viewed
Promoted


ElementClickInterceptedException: Message: element click intercepted: Element <div data-job-id="3642674545" class="display-flex job-card-container relative job-card-list
        job-card-container--clickable
        
        job-card-list--underline-title-on-hover  jobs-search-two-pane__job-card-container--viewport-tracking-3" data-view-name="job-card">...</div> is not clickable at point (232, 621). Other element would receive the click: <strong>...</strong>
  (Session info: chrome=125.0.6422.176)
Stacktrace:
	GetHandleVerifier [0x00007FF699A01F52+60322]
	(No symbol) [0x00007FF69997CEC9]
	(No symbol) [0x00007FF699837EBA]
	(No symbol) [0x00007FF69988F32E]
	(No symbol) [0x00007FF69988CCF2]
	(No symbol) [0x00007FF69988A18B]
	(No symbol) [0x00007FF699889356]
	(No symbol) [0x00007FF69987B491]
	(No symbol) [0x00007FF6998AC21A]
	(No symbol) [0x00007FF69987ADB6]
	(No symbol) [0x00007FF6998AC430]
	(No symbol) [0x00007FF6998CBC80]
	(No symbol) [0x00007FF6998ABFC3]
	(No symbol) [0x00007FF69987961